# Readable Cryptarithms Creator

<table rules=none><tr>

<td><p>
A term coined in 1931 by mathematician Simon Vatriquant (pictured right), a <a href="https://en.wikipedia.org/wiki/Verbal_arithmetic">cryptarithm</a> is an arithmetic puzzle in which letters have been substituted for numbers. Solving a cryptarithm involves finding all possible pairings of digits with letters that produce a numerically correct answer. Each letter represents a different digit, and the leading digit of each word cannot be zero.
<br><br>
This workbook uses a dictionary of English words (<a href="https://github.com/dwyl/english-words">hosted here</a>) to create three word cryptarithms.
</p></td>

<td><img src="https://upload.wikimedia.org/wikipedia/commons/d/d2/SimonVatriquant.JPG" alt="Vatriquant" style="width:250px;"/></td>
</tr></table>

#### Initialisation

In [3]:
import urllib.request, json # retrieve english dictionary
import itertools # permutations and combinations of digits
from IPython.display import clear_output # clear progress print statements
from tqdm import tqdm # progress bars for slow loops

In [139]:
# Read English wordlist to dictionary
wordlist_url = "https://raw.githubusercontent.com/dwyl/english-words/master/words_dictionary.json"
with urllib.request.urlopen(wordlist_url) as response:
    print(f"Reading data (size {round(int(response.getheader('content-length'))/int(1024**2), 1)} MB)...")
    data = json.load(response)
    print("Data successfully read.")

Reading data (size 6.5 MB)...
Data successfully read.


In [15]:
# Helper functions

def assign_possible_digits_to_words(list_of_words, allow_zero_leading_letter=False):
    word_set = set(''.join(list_of_words))
    num_digits = len(word_set)

    if len(word_set) > 10:
        raise ValueError(f"The words '{list_of_words}' has more than 10 characters, cannot assign digits uniquely.")

    digit_combinations = itertools.combinations(range(10), num_digits)
    sets = list(set(itertools.permutations(dc)) for dc in digit_combinations)
    
    digits_to_map_list = []
    for s in sets:
        digits_to_map_list.extend(s)

    dicts = []
    for digits_to_map in digits_to_map_list:
        dicts.append(dict(zip(word_set, digits_to_map)))

    # prune dicts with first letters assigned to zero if allow_zero_leading_letter is False (default behaviour)
    if not allow_zero_leading_letter:
        first_letters = [w[0] for w in list_of_words]
        dicts_pruned = []
        for d in dicts:
            try:    # if 0 is in values but does not have a key in first_letters
                if list(d.keys())[list(d.values()).index(0)] not in first_letters:
                    dicts_pruned.append(d)
            except: # if 0 is not in values
                dicts_pruned.append(d)
        dicts = dicts_pruned

    return dicts


def value_word(word, dictionary):
    v = ""
    for char in word:
        v += str(dictionary[char])
    return int(v)


def is_valid_three_words(list_of_three_words, operator='+', allow_zero_leading_letter=False):
    if operator not in ['+', '*', '-', '/']:
        raise ValueError(f"Operator '{operator}' is not valid (use ['+', '*', '-', '/']).")
    
    print('Finding all possible digit mappings for letters...', end='\r')
    digit_dict = assign_possible_digits_to_words(list_of_three_words, allow_zero_leading_letter)
    clear_output(wait=False)

    if operator == '+':
        for d in tqdm(digit_dict, desc="Checking sums", leave=False):
            if value_word(list_of_three_words[0], d) + value_word(list_of_three_words[1], d) == value_word(list_of_three_words[2], d):
                return True
    elif operator == '*':
        for d in tqdm(digit_dict, desc="Checking products", leave=False):
            if value_word(list_of_three_words[0], d) * value_word(list_of_three_words[1], d) == value_word(list_of_three_words[2], d):
                return True
    elif operator == '-':
        for d in tqdm(digit_dict, desc="Checking differences", leave=False):
            if value_word(list_of_three_words[0], d) - value_word(list_of_three_words[1], d) == value_word(list_of_three_words[2], d):
                return True
    elif operator == '/':
        for d in tqdm(digit_dict, desc="Checking quotients", leave=False):
            if value_word(list_of_three_words[0], d) / value_word(list_of_three_words[1], d) == value_word(list_of_three_words[2], d):
                return True
            
    return False

In [20]:
WORD1 = "SEND"
WORD2 = "MORE"
WORD3 = "MONEY"
OPERATOR = "+"

In [21]:
result = is_valid_three_words([WORD1, WORD2, WORD3], OPERATOR, allow_zero_leading_letter=False)
if result:
    print(f"{WORD1} {OPERATOR} {WORD2} = {WORD3} is a valid cryptarithm!")
else:
    print(f"{WORD1} {OPERATOR} {WORD2} = {WORD3} is not a valid cryptarithm.")

SEND + MORE = MONEY is a valid cryptarithm!
